<a href="https://colab.research.google.com/github/jjkiljanski/biebrza-shrub-encroachment-analysis/blob/main/gee_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q earthengine-api geemap

import ee
import geemap

# Log in to your Google account for Earth Engine (you will see a link)
ee.Authenticate()

# Start Earth Engine
ee.Initialize(project='biebrza-encroachment-analysis')

# Load Resources

In [2]:
import ipywidgets as widgets
from datetime import datetime

# ============================================================
# Load Biebrzański National Park boundary
#    (from the WDPA – World Database on Protected Areas)
# ============================================================

wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")

# Filter areas whose NAME contains "Biebrza" (safe way to match spelling)
bpn = wdpa.filter(ee.Filter.stringContains("NAME", "Biebrza"))

print("Number of matching park polygons:", bpn.size().getInfo())

# Geometry for clipping satellite images
region_geom = bpn.geometry()


Number of matching park polygons: 3


In [3]:
# ============================================================
# Load a shapefile defining 1997-2015 MPC change categories
#    derived from Kopeć and Sławik (2020)
# ============================================================

# FeatureCollection with ALL shrunk squares
mpc_all = ee.FeatureCollection(
    "projects/biebrza-encroachment-analysis/assets/MPC_all_traj_simple_97_15_shrunk25m"
)

# Name of the category column in the shapefile
category_prop = "traj_simpl"

# Get the list of unique categories from the shapefile (server-side)
categories = ee.List(mpc_all.aggregate_array(category_prop).distinct().sort())

# Bring list to client for looping / printing
category_list = categories.getInfo()
print("Categories found in shapefile:", category_list)

# --- DROP empty string categories ---
category_list = [c for c in category_list if c not in ("", None)]
print("Categories after removing empty / None:", category_list)

# Optional: build a dict of FeatureCollections keyed by category
mpc_fc = {
    cat: mpc_all.filter(ee.Filter.eq(category_prop, cat))
    for cat in category_list
}

# Example: use one category (if present)
# e.g. first category in the list
if category_list:
    example_cat = category_list[0]
    example_fc = mpc_fc[example_cat]
    print("Example category:", example_cat)
else:
    example_fc = None
    print("No categories found in shapefile.")

Categories found in shapefile: ['', 'shrubs_to_trees', 'stable_shrubs', 'stable_trees', 'stable_wetland', 'wetland_to_shrubs', 'wetland_to_trees']
Categories after removing empty / None: ['shrubs_to_trees', 'stable_shrubs', 'stable_trees', 'stable_wetland', 'wetland_to_shrubs', 'wetland_to_trees']
Example category: shrubs_to_trees


# Prepare the photo sequence

In [4]:
def get_landsat_l2_sr():
    lt5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')  # do 2011
    le7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')  # 1999–dzisiaj (SLC-off od 2003)
    lc8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')  # od 2013
    lc9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')  # od ~2021

    return lt5.merge(le7).merge(lc8).merge(lc9)

landsat = get_landsat_l2_sr()

In [5]:
# ================================================================
# Mask function for Landsat Collection 2 Level 2 Surface Reflectance
# ================================================================

def mask_l2_sr(image):
    """
    Applies the official USGS cloud/shadow/snow/saturation masks
    for Landsat Collection 2 Level-2 Surface Reflectance products.

    QA_PIXEL bits:
      Bit 0: Fill
      Bit 1: Dilated Cloud
      Bit 2: Cirrus
      Bit 3: Cloud
      Bit 4: Cloud Shadow

    QA_RADSAT:
      Indicates radiometric saturation in any band (values > 0 should be masked)
    """
    qa = image.select('QA_PIXEL')
    radsat = image.select('QA_RADSAT')

    # Bits 0–4 all must be zero (no fill, no clouds, no cirrus, no shadow)
    cloud_bits = int('11111', 2)
    qa_mask = qa.bitwiseAnd(cloud_bits).eq(0)

    # Radiometric saturation mask (value must be zero)
    sat_mask = radsat.eq(0)

    return image.updateMask(qa_mask).updateMask(sat_mask)

In [6]:
# =======================================================================
# Function to add NDMI, NBR and keep NIR as an index band for each image
# =======================================================================

def add_indices(image):
    # Work in float to avoid integer division issues
    image = image.toFloat()

    nir   = image.select('NIR')
    swir1 = image.select('SWIR1')
    swir2 = image.select('SWIR2')

    # NDMI = (NIR - SWIR1) / (NIR + SWIR1)
    ndmi = nir.subtract(swir1).divide(nir.add(swir1)).rename('NDMI')

    # NBR = (NIR - SWIR2) / (NIR + SWIR2)
    nbr = nir.subtract(swir2).divide(nir.add(swir2)).rename('NBR')

    # Add the index bands to each image
    return image.addBands([ndmi, nbr])

In [7]:
# ================================================================
# Correct band mapping for Landsat 5/7 (TM / ETM+)
# ================================================================

def prep_ls57(image):
    """
    Prepares Landsat 5 (TM) and Landsat 7 (ETM+) Level-2 SR images.

    Landsat 5/7 Surface Reflectance band names:
      SR_B1 = Blue
      SR_B2 = Green
      SR_B3 = Red
      SR_B4 = Near Infrared (NIR)
      SR_B5 = SWIR1
      SR_B7 = SWIR2
    """
    image = mask_l2_sr(image)

    # Rename bands to a unified spectral scheme (BLUE, GREEN, RED, NIR)
    return image.select(
        ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'],
        ['BLUE',  'GREEN', 'RED',  'NIR', 'SWIR1', 'SWIR2']
    )


# ================================================================
# Correct band mapping for Landsat 8/9 (OLI / OLI-2)
# ================================================================

def prep_ls89(image):
    """
    Prepares Landsat 8 and Landsat 9 Level-2 SR images.

    Landsat 8/9 Surface Reflectance band names:
      SR_B2 = Blue
      SR_B3 = Green
      SR_B4 = Red
      SR_B5 = Near Infrared (NIR)
      SR_B6 = SWIR1
      SR_B7 = SWIR2

    IMPORTANT:
      This mapping is DIFFERENT from Landsat 5/7!
      That’s why we cannot "guess" the mapping — we must explicitly map by sensor.
    """
    image = mask_l2_sr(image)

    return image.select(
        ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'],
        ['BLUE',  'GREEN', 'RED',  'NIR', 'SWIR1', 'SWIR2']
    )


# ================================================================
# Build a single harmonized image collection combining L5/7/8/9
# ================================================================

# Landsat TM (5)  → use prep_ls57
lt5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').map(prep_ls57)

# Landsat ETM+ (7) → use prep_ls57
le7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').map(prep_ls57)

# Landsat OLI (8)  → use prep_ls89
lc8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').map(prep_ls89)

# Landsat OLI-2 (9) → use prep_ls89
lc9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').map(prep_ls89)

# Unified L5+L7+L8+L9 collection with consistent band names
landsat = lt5.merge(le7).merge(lc8).merge(lc9)


# Add NDMI & NBR to each image, and keep only NDMI, NBR, NIR for analysis
landsat = (landsat
           .map(add_indices)
           .select(['NDMI', 'NBR', 'NIR']))

In [8]:
#-------------------------------------------------
# annual composite with per-pixel obs_count
#   + per-year quality summary properties
# -------------------------------------------------

def annual_composite(year, geom, scale=30):
    """
    Build an annual median composite (May–Sep) and attach:
      - 'obs_count' band: number of valid observations per pixel
      - properties summarizing obs_count (mean, std, min, max)
      - property 'obs_hist': histogram of obs_count over geom
      - property 'image_count': number of images in the collection
    """
    year = int(year)
    start = ee.Date.fromYMD(year, 5, 1)
    end   = ee.Date.fromYMD(year, 9, 30)

    col = (landsat
           .filterBounds(geom)
           .filterDate(start, end))

    # Number of images in the collection for this year (anywhere in geom)
    image_count = col.size()

    # Per-pixel count of valid (unmasked) observations
    # Use any band (e.g. NIR) as they are all masked consistently.
    obs_count = col.select('NIR').count().rename('obs_count')

    # Median composite of reflectance bands
    median_img = col.median()

    # Combine reflectance + obs_count
    img = median_img.addBands(obs_count).clip(geom)

    # Global stats of obs_count over the geometry
    # (mean, stdDev, min, max) – good quick QC indicators.
    obs_stats = obs_count.reduceRegion(
        reducer=ee.Reducer.mean()\
                    .combine(ee.Reducer.stdDev(), sharedInputs=True)\
                    .combine(ee.Reducer.minMax(), sharedInputs=True),
        geometry=geom,
        scale=scale,
        maxPixels=1e12
    )

    # Attach metadata as image properties
    img = img.set({
        'year': year,
        'image_count': image_count,
        'obs_mean':  obs_stats.get('obs_count_mean'),
        'obs_std':   obs_stats.get('obs_count_stdDev'),
        'obs_min':   obs_stats.get('obs_count_min'),
        'obs_max':   obs_stats.get('obs_count_max'),
    })

    return img

In [7]:
years = [1997,2015]#list(range(1997, 2026))
for y in years:
    img = annual_composite(y, region_geom)

    year        = img.get('year').getInfo()
    image_count = img.get('image_count').getInfo()
    obs_mean    = img.get('obs_mean').getInfo()
    obs_std     = img.get('obs_std').getInfo()
    obs_min     = img.get('obs_min').getInfo()
    obs_max     = img.get('obs_max').getInfo()

    print(f"\nYear {year}")
    print(f"  Images in collection       : {image_count}")
    print(f"  obs_count mean             : {obs_mean:.2f}")
    print(f"  obs_count std              : {obs_std:.2f}")
    print(f"  obs_count min / max        : {obs_min} / {obs_max}")


Year 1997
  Images in collection       : 35
  obs_count mean             : 8.87
  obs_count std              : 2.69
  obs_count min / max        : 2 / 18

Year 2015
  Images in collection       : 82
  obs_count mean             : 17.56
  obs_count std              : 4.85
  obs_count min / max        : 5 / 36


In [25]:
year = 1997
img = annual_images.filter(ee.Filter.eq('year', year)).first()

export_image = img.select(['RED','GREEN','BLUE','NIR','SWIR1','SWIR2']).toInt16()

task = ee.batch.Export.image.toDrive(
    image=export_image,
    description=f'biebrza_{year}_not_scaled',
    folder='GEE_Biebrza',
    fileNamePrefix=f'biebrza_{year}',
    scale=30,
    region=region_geom,
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)
task.start()

# Create pixel series

In [9]:
# ============================================================
# Compute the number of Landsat pixels falling into each category
# ============================================================

# -----------------------------------------------
# Get a proper 30 m Landsat projection
# -----------------------------------------------

# 'region_geom' must be defined earlier in your script
ref_img = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(region_geom) \
    .first()

# Use NIR band as reference (could be any band)
ref_proj = ref_img.select('SR_B5').projection()
pixel_scale = ref_proj.nominalScale()

print('Reference nominal scale (m):', pixel_scale.getInfo())

# -----------------------------------------------
# Choose an annual composite and band
# -----------------------------------------------

year_for_count = 2015  # change as you like

# Your existing function to build annual composite
annual_img = annual_composite(year_for_count, region_geom)

# Pick a band to define the pixel grid (e.g. NIR)
band_name = 'NIR'

# Reproject the composite band to the reference 30 m grid
band_img = annual_img.select(band_name).reproject(ref_proj)

# -----------------------------------------------
# Function: count pixels in each FC
#   (centroids inside polygons)
# -----------------------------------------------

def count_pixels_in_fc(image, fc, scale):
    """
    Counts unmasked pixels of `image` whose CENTROIDS
    fall inside each feature of `fc`.

    Returns the total count across all features.
    """
    counted_fc = image.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.count(),
        scale=scale
        # Default behavior: pixel counted if centroid is inside geometry
    )

    # Sum 'count' over all features in this FC
    total = counted_fc.aggregate_sum('count')
    return total

# -----------------------------------------------
# Loop over categories (from shapefile column)
# -----------------------------------------------

results = {}

for cat in category_list:
    fc_cat = mpc_fc[cat]  # filtered FeatureCollection for this category
    total_pixels = count_pixels_in_fc(band_img, fc_cat, pixel_scale).getInfo()
    results[cat] = total_pixels
    print(f"{cat}: {total_pixels} pixels (centroids inside polygons)")

# `results` now holds pixel counts for all categories found in the shapefile

Reference nominal scale (m): 30
shrubs_to_trees: 2736 pixels (centroids inside polygons)
stable_shrubs: 11369 pixels (centroids inside polygons)
stable_trees: 27490 pixels (centroids inside polygons)
stable_wetland: 42612 pixels (centroids inside polygons)


wetland_to_shrubs: 936 pixels (centroids inside polygons)
wetland_to_trees: 91 pixels (centroids inside polygons)


In [21]:
for cat in category_list:
  print(f"{cat}: {results[cat]} pixels (centroids inside polygons)")

In [ ]:
import geemap

# Choose a year to show as background (any year your annual_composite supports)
year_for_viz = 2015
viz_img = annual_composite(year_for_viz, region_geom)

# Simple RGB visualization
viz_params = {
    'bands': ['RED', 'GREEN', 'BLUE'],
    'min': 0.03,
    'max': 0.4,
}

# Create interactive map
m = geemap.Map()
m.centerObject(region_geom, 10)

# Add Landsat composite as background
m.addLayer(viz_img, viz_params, f'Landsat {year_for_viz} RGB')

# Add Biebrza NP boundary (wdpa / bpn from your script) as a thin outline
bpn_outline = bpn.style(color='white', width=2, fillColor='00000000')
m.addLayer(bpn_outline, {}, 'Biebrza NP (WDPA)')

# Add each category shapefile, styled differently
style_colors = {
    "stable_wetland":   '00FF00',  # green
    "wetland_to_shrubs": 'FFFF00', # yellow
    "wetland_to_trees":  'FF9900', # orange
    "stable_shrubs":     'FF00FF', # magenta
    "shrubs_to_trees":   '00FFFF', # cyan
    "stable_trees":      'FF0000', # red
}

for cat, fc in mpc_fc.items():
    color = style_colors.get(cat, 'FFFFFF')
    styled_fc = fc.style(color=color, width=2, fillColor='00000000')
    m.addLayer(styled_fc, {}, cat)

# Add layer control and display map
m.addLayerControl()
m

In [13]:
for cat in results.keys():
  print(f"{cat}: {total_pixels} pixels (centroids inside polygons)")

shrubs_to_trees: 0 pixels (centroids inside polygons)
stable_shrubs: 0 pixels (centroids inside polygons)
stable_trees: 0 pixels (centroids inside polygons)
stable_wetland: 0 pixels (centroids inside polygons)
wetland_to_shrubs: 0 pixels (centroids inside polygons)
wetland_to_trees: 0 pixels (centroids inside polygons)
